The original dataset from the reference consists of 5 different folders, each with 100 files, with each file representing a single subject/person. Each file is a recording of brain activity for 23.6 seconds. The corresponding time-series is sampled into 4097 data points. Each data point is the value of the EEG recording at a different point in time. So we have total 500 individuals with each has 4097 data points for 23.5 seconds.

Every 4097 data points were shuffled into 23 chunks, each chunk contains 178 data points for 1 second, and each data point is the value of the EEG recording at a different point in time. So now we have 23 x 500 = 11500 pieces of information(row), each information contains 178 data points for 1 second(column), the last column represents the label y {1,2,3,4,5}.

The response variable is y in column 179, the Explanatory variables X1, X2, ..., X178

y contains the category of the 178-dimensional input vector. Specifically y in {1, 2, 3, 4, 5}:

5 - eyes open, means when they were recording the EEG signal of the brain the patient had their eyes open

4 - eyes closed, means when they were recording the EEG signal the patient had their eyes closed

3 - Yes they identify where the region of the tumor was in the brain and recording the EEG activity from the healthy brain area

2 - They recorder the EEG from the area where the tumor was located

1 - Recording of seizure activity

All subjects falling in classes 2, 3, 4, and 5 are subjects who did not have epileptic seizure. Only subjects in class 1 have epileptic seizure. Our motivation for creating this version of the data was to simplify access to the data via the creation of a .csv version of it. Although there are 5 classes most authors have done binary classification, namely class 1 (Epileptic seizure) against the rest.

# Handling the Dataset

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [2]:
# Basic processing of dataframe
df_orig = pd.read_csv("data.csv")
columns = list(df_orig.columns)
columns.pop()
columns[0] = 'ID'
columns.append( 'Class' )
df_orig.columns = columns
df_orig = df_orig.set_index("ID")

Class = df_orig.Class
df = df_orig.drop(['Class'], axis = 1)

df

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
ID,,,,,,,,,,,,,,,,,,,,,
X21.V1.791,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
X15.V1.924,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,20,15,16,12,5,-1,-18,-37,-47,-48
X19.V1.354,-47,-11,28,77,141,211,246,240,193,136,...,-94,-65,-33,-7,14,27,48,77,117,170
X8.V1.28,14,6,-13,-16,10,26,27,-9,4,14,...,-42,-65,-48,-61,-62,-67,-30,-2,-1,-8


In [3]:
df_orig

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,Class
ID,,,,,,,,,,,,,,,,,,,,,
X21.V1.791,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
X15.V1.924,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,5
X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,-49,...,15,16,12,5,-1,-18,-37,-47,-48,2
X19.V1.354,-47,-11,28,77,141,211,246,240,193,136,...,-65,-33,-7,14,27,48,77,117,170,1
X8.V1.28,14,6,-13,-16,10,26,27,-9,4,14,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5


In [8]:
df = pd.read_csv (r'data.csv')
patient_ID = df["Unnamed: 0"]
time_data = [None for i in range(len(df))]
for i in range(len(patient_ID)):
    position_V = patient_ID[i].index('V')
    time_data[i] = int(patient_ID[i][1:position_V-1])
    patient_ID[i] = patient_ID[i][position_V+1:]
    patient_ID[i] = patient_ID[i].replace('.','0')
    patient_ID[i] = int(patient_ID[i])

df.insert(1, 'ID', patient_ID)
df.insert(0, 'Sequence', time_data)
df = df.drop(['Unnamed: 0'], axis=1)
df = df.sort_values(["y","ID","Sequence"])
df = df.set_index(['ID'])

/home/dell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/dell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/dell/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [9]:
ep=[]
for i in range(1,6):
    ep_1=df[df.y.isin([i])]
    ep.append(ep_1)
ep[3]

,Sequence,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
ID,,,,,,,,,,,,,,,,,,,,,
11,1,-24,-22,-17,-18,-19,-14,-4,-1,7,...,33,46,36,31,33,37,45,33,20,4
11,2,-1,-19,-30,-29,-33,-29,-28,-21,-14,...,-59,-41,-19,18,37,40,32,27,10,4
11,3,-11,-35,-64,-81,-90,-71,-48,-28,-14,...,26,31,39,46,46,48,48,54,46,4
11,4,54,43,38,18,-8,-27,-37,-8,11,...,-31,-17,-27,-28,-44,-27,2,19,26,4
11,5,17,6,-10,-21,-31,-44,-52,-68,-85,...,-97,-81,-64,-32,4,20,18,17,19,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10991,19,184,198,187,147,102,53,8,-40,-117,...,-71,-86,-85,-79,-36,5,53,83,86,4
10991,20,65,25,1,-5,-4,-6,-21,-33,-32,...,15,-36,-84,-119,-146,-171,-187,-166,-105,4
10991,21,-6,96,163,181,141,72,-19,-95,-131,...,-145,-162,-135,-75,-14,69,155,224,252,4


In [10]:
ep[3].index.value_counts()

10231    23
10601    23
10501    23
10511    23
10521    23
         ..
10931    23
10941    23
10951    23
10961    23
10241    23
Name: ID, Length: 100, dtype: int64

In [11]:
ep_df_agregated = []
for i in range(1,6):
    ep_new=ep[i-1].drop(['Sequence','y'], axis=1)
    ep_agg = pd.DataFrame([y.values.ravel() for x , y in ep_new.groupby(np.arange(len(ep_new))//23)])
    ep_agg.index = list(ep_new.index.unique())
    ep_agg["target"] = None
    for index in ep_agg.index:
        ep_agg.at[index,"target"] = int(ep[i-1].at[index,"y"].unique())
    ep_df_agregated.append(ep_agg)

In [12]:
ep_df_agregated[4]["target"].value_counts()

5    100
Name: target, dtype: int64

In [38]:
#Aggregated Data Frame: Each row is a Patient
df_agg = pd.DataFrame()
for i in ep_df_agregated:
    df_agg = pd.concat([df_agg,i])
df_agg

,0,1,2,3,4,5,6,7,8,9,...,4085,4086,4087,4088,4089,4090,4091,4092,4093,target
14,100,124,153,185,210,220,216,222,240,265,...,478,378,250,137,57,17,0,7,26,1
10104,142,262,382,451,452,435,544,562,462,182,...,164,-2,-118,-223,-304,-350,-339,-299,-253,1
10116,340,353,400,470,538,590,611,629,649,693,...,-75,-18,205,556,824,919,916,829,722,1
10117,457,420,344,272,198,122,35,-69,-168,-258,...,-121,-135,-136,-149,-152,-180,-209,-237,-247,1
10124,746,692,562,400,238,84,-36,-137,-211,-253,...,-80,-251,-506,-776,-989,-853,-593,-89,310,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,-133,-144,-113,-74,-41,-41,-61,-89,-107,-115,...,-115,-103,-88,-81,-70,-58,-62,-71,-79,5
1096,-3,-3,-12,-24,-34,-42,-41,-49,-20,7,...,-40,-41,-28,-4,31,54,61,55,42,5
1097,-51,-42,-39,-47,-51,-46,-29,-7,9,10,...,45,56,46,23,-7,-16,-18,-9,2,5
1098,56,55,38,-5,-47,-72,-79,-62,-39,-21,...,-53,-68,-44,-3,36,41,14,-27,-45,5


# Implementing LSTM Neural Network Model

An LSTM network is a recurrent neural network consisting of LSTM layers having the ability to selectively remember important information for a longer period and is mostly used for sequence prediction. Since EEG signal is a time series, the sequence is important and has to be considered.

## Binary Classification 

In [14]:
X = df.iloc[:,1:179].values.T
X.shape

(178, 11500)

In [15]:
y = df.iloc[:,179].values
y

array([1, 1, 1, ..., 5, 5, 5])

To make this a binary problem, let's turn the non-seizure classes 0 while maintaining the seizure as 1.

In [16]:
y[y>1]=0
y

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
# Normalising X data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
x = scaler.transform(X).T
from tensorflow.keras.utils import to_categorical

#One Hot Encoding y
# y = to_categorical(y)
# y.shape

In [18]:
y = np.expand_dims(y,1)
x.shape, y.shape

((11500, 178), (11500, 1))

In [19]:
#Train Test Split
from sklearn.model_selection import train_test_split, cross_val_score
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [20]:
x_train = np.expand_dims(x_train,2)
x_test = np.expand_dims(x_test,2)
x_train.shape, y_train.shape

((9200, 178, 1), (9200, 1))

In [21]:
#Model Architecture
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import SGD

tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(178,1),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid"))
model.add(Dropout(0.5))

# Since Binary Classification:
model.add(Dense(1, activation='sigmoid'))


callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True)
]
model.compile(
    optimizer = tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy']
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 178, 64)           16896     
_________________________________________________________________
dropout (Dropout)            (None, 178, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 29,345
Trainable params: 29,345
Non-trainable params: 0
_________________________________________________________________


In [22]:
history = model.fit(x_train, y_train, epochs=20, validation_data= (x_test, y_test), callbacks=callbacks)
score, acc = model.evaluate(x_test, y_test)

Epoch 1/20
288/288 [==============================] - 54s 174ms/step - loss: 0.5156 - accuracy: 0.7764 - val_loss: 0.4684 - val_accuracy: 0.8026
Epoch 2/20
288/288 [==============================] - 52s 179ms/step - loss: 0.5222 - accuracy: 0.7958 - val_loss: 0.4932 - val_accuracy: 0.8026
Epoch 3/20
288/288 [==============================] - 52s 182ms/step - loss: 0.5185 - accuracy: 0.7993 - val_loss: 0.5179 - val_accuracy: 0.8026
Epoch 4/20
288/288 [==============================] - 54s 189ms/step - loss: 0.5166 - accuracy: 0.7993 - val_loss: 0.4969 - val_accuracy: 0.8026
Epoch 5/20
288/288 [==============================] - 55s 192ms/step - loss: 0.5118 - accuracy: 0.7993 - val_loss: 0.4969 - val_accuracy: 0.8026
Epoch 6/20
288/288 [==============================] - 54s 188ms/step - loss: 0.5102 - accuracy: 0.7993 - val_loss: 0.4972 - val_accuracy: 0.8026
Epoch 7/20
288/288 [==============================] - 45s 155ms/step - loss: 0.5090 - accuracy: 0.7993 - val_loss: 0.4969 - val_ac

The accuracy of binary classification from the above LSTM model is around 0.8, which is not great, considering the fact that the class split is 4:1, implying that the model is similar to that of randomly guessing.

## Multi Class Classification

In [24]:
# Standardisation of X data
X = df_orig.iloc[:,:178].values.T
scaler = StandardScaler()
scaler.fit(X)
x = scaler.transform(X).T
x.shape

(11500, 178)

In [25]:
y = df_orig.iloc[:,178].values
y

array([4, 1, 5, ..., 5, 3, 4])

In [26]:
# Mapping y:[1,5] to y:[0,4]
y_one_hot = []
for num in y:
    y_one_hot.append(num-1)
y_one_hot

[3,
 0,
 4,
 4,
 4,
 4,
 3,
 1,
 0,
 3,
 4,
 0,
 2,
 3,
 1,
 2,
 1,
 2,
 3,
 1,
 0,
 1,
 0,
 0,
 0,
 2,
 4,
 0,
 1,
 1,
 0,
 1,
 3,
 4,
 4,
 2,
 3,
 2,
 4,
 2,
 4,
 1,
 0,
 3,
 2,
 1,
 3,
 1,
 4,
 2,
 4,
 2,
 0,
 2,
 2,
 4,
 2,
 4,
 0,
 2,
 2,
 2,
 3,
 4,
 4,
 0,
 3,
 1,
 0,
 2,
 3,
 2,
 3,
 3,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 0,
 0,
 1,
 1,
 4,
 0,
 3,
 2,
 3,
 1,
 3,
 2,
 4,
 1,
 2,
 3,
 2,
 4,
 3,
 0,
 3,
 3,
 1,
 4,
 0,
 2,
 3,
 2,
 2,
 2,
 3,
 1,
 0,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 3,
 0,
 1,
 1,
 4,
 4,
 1,
 2,
 1,
 0,
 2,
 4,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 4,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 4,
 2,
 4,
 2,
 1,
 1,
 0,
 3,
 3,
 0,
 0,
 4,
 3,
 3,
 0,
 0,
 4,
 3,
 4,
 0,
 3,
 0,
 4,
 4,
 0,
 4,
 0,
 1,
 0,
 0,
 2,
 3,
 4,
 0,
 4,
 1,
 4,
 1,
 2,
 3,
 0,
 4,
 3,
 4,
 4,
 1,
 2,
 2,
 3,
 2,
 0,
 1,
 3,
 0,
 3,
 1,
 4,
 4,
 0,
 4,
 0,
 2,
 1,
 3,
 3,
 4,
 1,
 2,
 3,
 0,
 3,
 0,
 0,
 0,
 2,
 4,
 2,
 0,
 2,
 4,
 0,
 1,
 3,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 3,
 4,
 0,
 0,
 4,
 4,
 0,
 2,


In [27]:
#One Hot Encoding y
y_one_hot= to_categorical(y_one_hot)
y_one_hot

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [28]:
#Train Test Split
x_train, x_test, y_train, y_test = train_test_split(x, y_one_hot, test_size = 0.2)
x_train = np.expand_dims(x_train,2)
x_test = np.expand_dims(x_test,2)
print(x_train.shape)
print(y_train.shape)


(9200, 178, 1)
(9200, 5)


In [29]:
#Setting variables
epochs = 100
batch_size = 256

In [30]:
#Model Architecture
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(178,1),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="relu"))
model.add(Dropout(0.2))

# Multiclass Classification so last layer is softmax
model.add(Dense(5,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 178, 64)           16896     
_________________________________________________________________
dropout (Dropout)            (None, 178, 64)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 29,477
Trainable params: 29,477
Non-trainable params: 0
_________________________________________________________________


In [31]:
#Adding Early Stopping so that the model stops running if val_loss is increasing over some epochs 
from keras.callbacks import EarlyStopping

#Fit the model
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001, restore_best_weights=True)])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/100
29/29 [==============================] - 14s 433ms/step - loss: 135.3327 - accuracy: 0.2149 - val_loss: 74568184.0000 - val_accuracy: 0.2168
Epoch 2/100
29/29 [==============================] - 12s 396ms/step - loss: 5881837568.0000 - accuracy: 0.2285 - val_loss: 277.9197 - val_accuracy: 0.2304
Epoch 3/100
29/29 [==============================] - 13s 441ms/step - loss: nan - accuracy: 0.2272 - val_loss: nan - val_accuracy: 0.2158
Epoch 4/100
29/29 [==============================] - 12s 411ms/step - loss: nan - accuracy: 0.2027 - val_loss: nan - val_accuracy: 0.2158
Epoch 5/100
29/29 [==============================] - 11s 392ms/step - loss: nan - accuracy: 0.2027 - val_loss: nan - val_accuracy: 0.2158
Epoch 6/100
29/29 [==============================] - 11s 392ms/step - loss: nan - accuracy: 0.2027 - val_loss: nan - val_accuracy: 0.2158
Epoch 7/100
29/29 [==============================] - 12s 405ms/step - loss: nan - accuracy: 0.2027 - val_loss: nan - val_accuracy: 0.2158
Ep

The accuracy of multi class classification from the above LSTM model is around 0.25, which is not great, considering the fact that the class split is evenly amongst the five classes, implying that the model is similar to that of randomly guessing.

## Considering Each Patient as Input

In [32]:
# Standardisation of X data
from sklearn.preprocessing import StandardScaler
X = df_agg.iloc[:,:4095].values.T
scaler = StandardScaler()
scaler.fit(X)
x = scaler.transform(X).T
x.shape

(500, 4095)

In [33]:
from tensorflow.keras.utils import to_categorical
y = df_agg['target'].values

# Mapping y:[1,5] to y:[0,4]
y_one_hot = []
for num in y:
    y_one_hot.append(num-1)

#One Hot Encoding y
y_one_hot= to_categorical(y_one_hot)
y_one_hot.shape

(500, 5)

In [34]:
#Train Test Split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y_one_hot, test_size = 0.2)
x_train = np.expand_dims(x_train,2)
x_test = np.expand_dims(x_test,2)
print(x_train.shape)
print(y_train.shape)

(400, 4095, 1)
(400, 5)


In [35]:
#Model Architecture
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import SGD
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(64, input_shape=(4095,1),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="relu"))
model.add(Dropout(0.2))

# Multiclass Classification so last layer is softmax
model.add(Dense(5,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4095, 64)          16896     
_________________________________________________________________
dropout (Dropout)            (None, 4095, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 165       
Total params: 29,477
Trainable params: 29,477
Non-trainable params: 0
_________________________________________________________________


In [36]:
#Setting variables
epochs = 100
batch_size = 256

In [37]:
#Adding Early Stopping so that the model stops running if val_loss is increasing over some epochs 
from keras.callbacks import EarlyStopping

#Fit the model
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001, restore_best_weights=True)])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/100
2/2 [==============================] - 21s 6s/step - loss: 1.6093 - accuracy: 0.1969 - val_loss: 1.6100 - val_accuracy: 0.2500
Epoch 2/100
2/2 [==============================] - 16s 6s/step - loss: 1.6076 - accuracy: 0.2094 - val_loss: 1.6100 - val_accuracy: 0.2750
Epoch 3/100
2/2 [==============================] - 16s 6s/step - loss: 1.6078 - accuracy: 0.2250 - val_loss: 1.6092 - val_accuracy: 0.2625
Epoch 4/100
2/2 [==============================] - 17s 6s/step - loss: 1.6065 - accuracy: 0.2438 - val_loss: 1.6085 - val_accuracy: 0.2875
Epoch 5/100
2/2 [==============================] - 16s 6s/step - loss: 1.6067 - accuracy: 0.2313 - val_loss: 1.6077 - val_accuracy: 0.3000
Epoch 6/100
2/2 [==============================] - 17s 6s/step - loss: 1.6022 - accuracy: 0.2688 - val_loss: 1.6070 - val_accuracy: 0.2625
Epoch 7/100
2/2 [==============================] - 16s 6s/step - loss: 1.6034 - accuracy: 0.2688 - val_loss: 1.6063 - val_accuracy: 0.2125
Epoch 8/100
2/2 [==========

Conclusion: Considering each patient as input for the neural network did not improve the model and actually resulted in a decrease of accuracy to 21%, just about what would be if one randoml guessed.